In [2]:
import numpy as np
import pandas as pd
from yelpapi import YelpAPI
from googleplaces import GooglePlaces, types, lang
import googlemaps 
import gmaps as maps

In [3]:
# Open Yelp API key file and read key
# The file is assumed to contain only one single line with the API key

def read_file_content(filename):
    """ Given a filename,
        returns the content of this file
    """
    try:
        with open(filename, 'r') as f:
            return f.read()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

YELP_API_KEY = read_file_content("./API Keys/YELP_API_KEY.txt")
yelp_api = YelpAPI(YELP_API_KEY, timeout_s=3.0)

In [4]:
# Configure Google Places
PLACES_API_KEY = read_file_content("./API Keys/GOOGLE_PLACES_API_KEY.txt")
gmaps = googlemaps.Client(key=PLACES_API_KEY)

### Yelp Top 20 vs. Google Top 20 Restaurants in Stuttgart

Comparing the Top 20 results for Restaurants near Stuttgart from Google vs. Yelp, it seems that the Google ranking tends to prefer high-quality restaurants while the Yelp ranking tends to prefer restaurants visited frequently (e.g. 'Frittenwerk' vs. 'Hotel Royal', 'Biertargen im Schlossgarten' vs. 'Der Zauberlehrling')

In [5]:
# Lat/lng Stuttgart Hbf
lat_hbf = 48.783333
lng_hbf = 9.183333

In [6]:
# Top 20 restaurants in Stuttgart
response = yelp_api.search_query(term='restaurant', latitude=lat_hbf, longitude=lng_hbf, sort_by='best_match', limit=20)

In [7]:
def print_restaurant_names(response):
    restaurants = response['businesses']
    for rest in restaurants:
        print (rest['name'])
        
def get_restaurant_names(response):
    restaurant_list = []
    restaurants = response['businesses']
    for rest in restaurants:
        restaurant_list.append(rest['name'])
    return restaurant_list

In [8]:
print_restaurant_names(response)

Frittenwerk
Biergarten im Schlossgarten
Poffers Café Stuttgart
Gaststätte Schlesinger
Die Zirbelstube
Kleinschmeckerei
Takeshii's
Aspendos
Ramen 8
Gravity
Zum Becher
Italiani
Food Court
L'Osteria
Mandu
Yorma's
Rhythms Bar & Kitchen
Cube
Brauhaus Schönbuch
Injeera


In [11]:
# Compare Top 20 Google results with top 20 Yelp results -> there is no common element
top_20_yelp = get_restaurant_names(response)

# Google Top 20, which we recieved earlier (but saved only the IDs) 
top_20_google = ['Hotel Royal', 'Der Zauberlehrling', 'Alte Kanzlei', 'Oggi', 'Enchilada Stuttgart', 
                 '5 Bar Gourmetrestaurant', 'Valle', 'Sky Beach Stuttgart', 'BLOCK HOUSE Eberhardstraße', 
                 'Arche Weinstube', 'VAPIANO Suttgart Bolzstrasse', 'Weinstube Fröhlich', 
                 'Paulaner am alten Postplatz', 'Prince of India', 'Hotel-Restaurant Köhler', 'Ambiente Africa', 
                 'MAREDO Steakhaus Stuttgart', 'BLOCK HOUSE Arnulf-Klett-Platz', 'CUBE Restaurant', 'Amici']

list(set(top_20_yelp) & set(top_20_google))

[]

### Download Yelp IDs based on Multiple Google Places Nearby Searches

Earlier we downloaded a list of ~1300 Google Place IDs for the most popular restaurants in Stuttgart area. The list was obtained by making multiple Google Places Nearby Searches around varying search centers. This list of IDs represents a good selection of restaurants (in terms of relevancy and quality of the restaurants), and the starting point for our Restaurant database.

Since we are not allowed to save any Google Places data other than the ID, we need to gather the restaurant information from the Yelp API, matching the Google Places ID to the corresponding Yelp IDs.

To do so, we first look up name and location at Google Places for each Google ID, then try to find the corresponding Yelp business by making a Yelp search_query based on name and location.

In [13]:
# For each restaurant id in the Google list, lookup name and geolocation at Google,
# then do a matching query at Yelp

google_restaurant_ids = np.array(
    pd.read_csv('./data/google_restaurant_ids.csv')['id'])

In [19]:
# For each Google ID, look up lat, lng, and name at Google Places
# Try to find matching Yelp Business based on name and geolocation.

yelp_ids_based_on_google = []
for google_id in google_restaurant_ids:
    print("Find Yelp ID for Google ID: {}".format(google_id))
    try:
        # Lookup name + geolocations at Google by ID 
        google_restaurant = gmaps.place(place_id=google_id)
        lat = google_restaurant['result']['geometry']['location']['lat']
        lng = google_restaurant['result']['geometry']['location']['lng']
        name = google_restaurant['result']['name']
        print("...restaurant {} at {}, {}".format(name, lat, lng))
        # Now try to find it at Yelp based on name and geolocation
        yelp_restaurant = yelp_api.search_query(term=name, latitude=lat, longitude=lng, radius=500, limit=2)
        #print(yelp_restaurant)
        yelp_id = yelp_restaurant['businesses'][0]['id']
        #print(yelp_restaurant['businesses'][0]['coordinates'])
        yelp_latitude = yelp_restaurant['businesses'][0]['coordinates']['latitude']
        yelp_longitude = yelp_restaurant['businesses'][0]['coordinates']['longitude']
        yelp_ids_based_on_google.append(yelp_id)
        print("...Yelp ID: {}".format(yelp_id))
    except Exception:
        try:
            print("No match found for Google name: {}".format(name))
        except Exeption:
            print("No match found for Google id: {}.format(google_id)")
        continue

Find Yelp ID for Google ID: ChIJdQ5X5XzPmUcRv7wL1Ond7Go
No match found for Google name: Daily Burger
Find Yelp ID for Google ID: ChIJCyT7XoDPmUcRVlr-BH_bNw8
...restaurant Restaurant "La Perla" im Schießsport-Zentrum at 48.8333999, 9.249574
No match found for Google name: Restaurant "La Perla" im Schießsport-Zentrum
Find Yelp ID for Google ID: ChIJleGdJZDJmUcRqozTDacuv14
...restaurant Andys Crepes at 48.8319935, 9.258655899999999
No match found for Google name: Andys Crepes
Find Yelp ID for Google ID: ChIJjzY04XzFmUcRkoCkdy647Co
...restaurant Restaurant TSV Steinhaldenfeld at 48.82968539999999, 9.2403472
...Yelp ID: -k0Ksk_FSzQAiKU6tdjayg
Find Yelp ID for Google ID: ChIJbc5V5XzPmUcRHvMxfGR2Afw
...restaurant Oettingers Restaurant at 48.8340312, 9.2634533
...Yelp ID: -lWPadzpW-DFB0C6UHvX6w
Find Yelp ID for Google ID: ChIJqxZJe5vFmUcRl6JkPcV7kGk
...restaurant Die Früchtle vom Schmidener Feld at 48.81933780000001, 9.254840100000001
...Yelp ID: 7Qvx3vZzuTNtB7kbCbQqZA
Find Yelp ID for Google 

No match found for Google name: Pizza Trullo‘s
Find Yelp ID for Google ID: ChIJM5ZhX4zPmUcRwLzsyq0PDmI
...restaurant Arschlöchle at 48.8398046, 9.232103100000002
No match found for Google name: Arschlöchle
Find Yelp ID for Google ID: ChIJL-TuR_bPmUcRDA0wgxOZTjI
...restaurant Treffpunkt am See at 48.83564000000001, 9.21872
...Yelp ID: noXKdbLQqpZPAiesHqPzdw
Find Yelp ID for Google ID: ChIJq0yCCtfEmUcRJeLQt23fGOY
...restaurant Taverne Timbales at 48.8413185, 9.2357325
No match found for Google name: Taverne Timbales
Find Yelp ID for Google ID: ChIJB52jMvPPmUcRAE__uR8-qqU
...restaurant Grillstation at 48.84043639999999, 9.2290019
No match found for Google name: Grillstation
Find Yelp ID for Google ID: ChIJYZpz2I3PmUcRmxwTc6Y-AuQ
...restaurant Stuttgart-Hofen at 48.84128, 9.23577
No match found for Google name: Stuttgart-Hofen
Find Yelp ID for Google ID: ChIJYZpz2I3PmUcRGNHLiqVQjlg
...restaurant Vereinsgaststätte SKG Stuttgart at 48.84128, 9.23577
No match found for Google name: Vereinsgas

No match found for Google name: Am Eck
Find Yelp ID for Google ID: ChIJMSQ6N_zPmUcRqGfXcDTU46M
...restaurant Bäckerei Böhm GmbH at 48.8416691, 9.2083277
No match found for Google name: Bäckerei Böhm GmbH
Find Yelp ID for Google ID: ChIJ40x44gPQmUcR1YX2QW2foN0
...restaurant Gasthaus Krone at 48.8410636, 9.1981596
No match found for Google name: Gasthaus Krone
Find Yelp ID for Google ID: ChIJt4_uNUrFmUcRv6_SqW5xbLo
...restaurant Maxs Gastronomie GmbH at 48.81775, 9.2008
No match found for Google name: Maxs Gastronomie GmbH
Find Yelp ID for Google ID: ChIJlZCHLMnbmUcRtH_jgDKRYG0
...restaurant Istanbul Kebab at 48.8317786, 9.1867054
No match found for Google name: Istanbul Kebab
Find Yelp ID for Google ID: ChIJBfOl5KPbmUcRMckqqqaAfeM
...restaurant مطعم الجنة at 48.8304524, 9.1864378
No match found for Google name: مطعم الجنة
Find Yelp ID for Google ID: ChIJVVVljaXamUcR_CMrFL_4ero
...restaurant Tanzcafe at 48.8304524, 9.1864378
No match found for Google name: Tanzcafe
Find Yelp ID for Googl

No match found for Google name: Aslan Kebap-Haus
Find Yelp ID for Google ID: ChIJadP-RYTamUcRvcG5DMKbDJM
...restaurant Pasquale Pepe Gaststätte at 48.8322519, 9.157652299999999
No match found for Google name: Pasquale Pepe Gaststätte
Find Yelp ID for Google ID: ChIJqWPHA8DamUcRhCIhw0xyS30
...restaurant Göktaş Restaurant at 48.81748, 9.170589999999999
No match found for Google name: Göktaş Restaurant
Find Yelp ID for Google ID: ChIJF5x_csDamUcRUf1WjH0ecew
...restaurant Köfteci Tuncay Stuttgart at 48.8171451, 9.1717659
No match found for Google name: Köfteci Tuncay Stuttgart
Find Yelp ID for Google ID: ChIJs2olT5namUcRx2SmpVXHGcw
...restaurant Fäßle at 48.8328068, 9.1697089
No match found for Google name: Fäßle
Find Yelp ID for Google ID: ChIJ47Ev1p3amUcR6nwe3JHMOzw
...restaurant Weinstube zum Tröpfle at 48.83598509999999, 9.169606
...Yelp ID: mZ_q2M-TvBJe2DLKvkoRmQ
Find Yelp ID for Google ID: ChIJh2q_Y_PbmUcRgtCdcrV1u20
...restaurant Mersin Tantuni at 48.82982599999999, 9.17067229999999

No match found for Google name: Vereinsheim Dietel
Find Yelp ID for Google ID: ChIJbQLj4kfbmUcRQS-gV5qk4JQ
...restaurant Indian Grill Station at 48.83704710000001, 9.149160400000001
No match found for Google name: Indian Grill Station
Find Yelp ID for Google ID: ChIJi63cq3_amUcRRnPCNxTeP94
...restaurant BIG BABA’s Kebabhouse at 48.83769139999999, 9.1477416
No match found for Google name: BIG BABA’s Kebabhouse
Find Yelp ID for Google ID: ChIJVRqHNPzbmUcRngld6xkc3DA
...restaurant Miroslav at 48.8301201, 9.136284300000002
No match found for Google name: Miroslav
Find Yelp ID for Google ID: ChIJb_Y6H3ramUcRlIAwzg-LxSk
...restaurant Rustical Vereinsheim der Gartenfreunde Grenzäcker at 48.83537339999999, 9.138789
No match found for Google name: Rustical Vereinsheim der Gartenfreunde Grenzäcker
Find Yelp ID for Google ID: ChIJBWy913HamUcRUkvrjkJYyJA
...restaurant Milano Lieferservice at 48.831228, 9.131924900000001
No match found for Google name: Milano Lieferservice
Find Yelp ID for Google I

...

...Yelp ID: 03RdFNkAXISiQ6sLndNRVg
Find Yelp ID for Google ID: ChIJ46pkTlrcmUcRfp0YhSSOu0M
...restaurant Harmonie Pizza Service at 48.7338454, 9.1084569
...Yelp ID: 81FR9Ncu_i9E8LBR3jh19g
Find Yelp ID for Google ID: ChIJq8i2M1zcmUcRlVKA4UDicDY
...restaurant Debitel QG Stuttgart at 48.7388055, 9.111433499999999
No match found for Google name: Debitel QG Stuttgart
Find Yelp ID for Google ID: ChIJj-edQFzcmUcRlTHHLGToBPQ
...restaurant TimeOut at 48.7377668, 9.110222199999999
No match found for Google name: TimeOut
Find Yelp ID for Google ID: ChIJq6rqG0LcmUcRg4w5SzgppWA
...restaurant SocialMania at 48.7415131, 9.1010301
No match found for Google name: SocialMania
Find Yelp ID for Google ID: ChIJidN06lfcmUcRDSHbSuqEFYQ
...restaurant Dong Kinh at 48.7299191, 9.1146959
...Yelp ID: cbqamMEVG5rPw4iSo6gGEQ
Find Yelp ID for Google ID: ChIJb8HIUVfcmUcR2u4hf1sZzz8
...restaurant Ya Ya Sushi at 48.7309915, 9.110281900000002
...Yelp ID: PuZNdoqk9XqGPjMeqqr_1g
Find Yelp ID for Google ID: ChIJX8BsAkLcmUc

No match found for Google name: Botticelli Food and More
Find Yelp ID for Google ID: ChIJeUjbwFDcmUcRZef-NDO0vdg
...restaurant asiahung at 48.73032800000001, 9.108488900000001
No match found for Google name: asiahung
Find Yelp ID for Google ID: ChIJlR_3ceDdmUcRa6Zuwh55UWg
...restaurant Lavanta Cafe&Bäckerei at 48.73007690000001, 9.103864500000002
No match found for Google name: Lavanta Cafe&Bäckerei
Find Yelp ID for Google ID: ChIJJZ8DgrPdmUcRv97o3BWmqmM
...restaurant Kebab Hütte Vaihingen Stuttgart Döner Pide Pizza Seele Drehspieß (Scheibenfleisch am Dienstag, Donnerstag, Samstag, Sonntag), (Hänchendöner am Montag, Mittwoch, Freitag), (Iskenderkebab am Donnerstag, Samstag, Sonntag) at 48.7300252, 9.1053864
No match found for Google name: Kebab Hütte Vaihingen Stuttgart Döner Pide Pizza Seele Drehspieß (Scheibenfleisch am Dienstag, Donnerstag, Samstag, Sonntag), (Hänchendöner am Montag, Mittwoch, Freitag), (Iskenderkebab am Donnerstag, Samstag, Sonntag)
Find Yelp ID for Google ID: ChIJ

No match found for Google name: Deutsch Cantina
Find Yelp ID for Google ID: ChIJSYImdg7DmUcR_K5_TUMnOAM
...restaurant Hunt Brothers Pizza at 48.72382650000001, 9.1761602
No match found for Google name: Hunt Brothers Pizza
Find Yelp ID for Google ID: ChIJPwaPe9PcmUcRLTe67mDKn2g
...restaurant SI-SUITES at 48.7239646, 9.161900500000002
...Yelp ID: czvBbsA02qiCyF5JlZPzfA
Find Yelp ID for Google ID: ChIJdX92ryzDmUcRT7PTos_X6GM
...restaurant Hotel Dormero at 48.7224317, 9.163261799999999
...Yelp ID: cnpyL8mCSEmjAC4QvRrApA
Find Yelp ID for Google ID: ChIJPwaPe9PcmUcRnjyOLuYe8mM
...restaurant Braustube Schlossturm at 48.7239092, 9.163105600000002
...Yelp ID: 6tR-uUoy8CScXuSRMw2igQ
Find Yelp ID for Google ID: ChIJPwaPe9PcmUcRqBqmQQYtFGk
...restaurant Red Grill Stuttgart at 48.72234520000001, 9.163226800000002
No match found for Google name: Red Grill Stuttgart
Find Yelp ID for Google ID: ChIJAfXM79PcmUcRScH0rV2UFHM
...restaurant Imbiss Inn at 48.72316389999999, 9.159522199999998
...Yelp ID: IEi

...Yelp ID: tdsVbIHvwPWFO764eWArvw
Find Yelp ID for Google ID: ChIJMxBCYifdmUcRjaIC5kvSIHc
...restaurant feinkost squisito at 48.71844, 9.146440000000002
No match found for Google name: feinkost squisito
Find Yelp ID for Google ID: ChIJv61J2dzcmUcRlxUdrJgyU-A
...restaurant Möhringer Pizza und Kebabhaus at 48.72787999999999, 9.144480000000001
No match found for Google name: Möhringer Pizza und Kebabhaus
Find Yelp ID for Google ID: ChIJZ7SxB9TdmUcRt4RGtDrdUUc
...restaurant Il Vulcano Ristorante Pizzeria at 48.72762729999999, 9.144417599999999
No match found for Google name: Il Vulcano Ristorante Pizzeria
Find Yelp ID for Google ID: ChIJhQtFi9zcmUcRwsLHnXp8S1M
...restaurant Momentos at 48.726394, 9.1453749
...Yelp ID: cJNw0OmXqYHumZsFFKtZFw
Find Yelp ID for Google ID: ChIJc7ND_DfdmUcR35dusO7jnUY
...restaurant Gaumenschmaus Haenchen at 48.7272853, 9.1413418
No match found for Google name: Gaumenschmaus Haenchen
Find Yelp ID for Google ID: ChIJqZzRkufcmUcR-_0a_XpPEV8
...restaurant Mediterra

No match found for Google name: Ivica Rezo Gaststättenbetriebe e.K.
Find Yelp ID for Google ID: ChIJVVUVqFfcmUcRORZYWoSeYyg
...restaurant Gülnaz Döner & Pizza at 48.7293539, 9.112712499999997
No match found for Google name: Gülnaz Döner & Pizza
Find Yelp ID for Google ID: ChIJf_1x5AvdmUcRD8Jh7REi_RM
...restaurant Eat Happy at 48.72962030000001, 9.1118479
...Yelp ID: 9A-y0TWBKqqP5vsn_CwI3g
Find Yelp ID for Google ID: ChIJB3Ey7FXcmUcRvQ2act8k2xk
...restaurant Aqua, Restaurant Bar und Lounge at 48.7251666, 9.1134439
No match found for Google name: Aqua, Restaurant Bar und Lounge
Find Yelp ID for Google ID: ChIJ0SO7gFbcmUcR91-ecxAdfhc
...restaurant FONTANA RESTAURANT at 48.7263747, 9.111349299999999
...Yelp ID: sj18RDxKq6n_JMYIJ8T4zg
Find Yelp ID for Google ID: ChIJib05d3vdmUcRyllsVjiP2fU
...restaurant Der Nudelstrauss ,,Lé Bouquet dé Pates" at 48.7302606, 9.111054700000002
No match found for Google name: Der Nudelstrauss ,,Lé Bouquet dé Pates"
Find Yelp ID for Google ID: ChIJK50s8iTdmUcR9

In [18]:
# Write to CSV file and Excel file
df = pd.DataFrame(yelp_ids_based_on_google, columns=['yelp_id'])
df.to_csv('yelp_ids_based_on_google_search.csv')
df.to_excel('yelp_ids_based_on_google_search.xlsx')

### Download Yelp IDs based on Multiple Yelp searches 

Only ~420 of the restaurants found via Google Places could be matched to a Yelp ID based on name and geolocation search. This is probably the case when the names at Google Places vs. Yelp differ too much (even though the Yelp API does not seem to require an exact match), or when the geolocations differ too much. However, we saw the results changing across multiple executions (cf. https://github.com/Yelp/yelp-fusion/issues/197), and the number of matches ranged between 408 and 430.

So we need to supplement these matches with additional results, retrieved from Yelp directly. 

Even though the Yelp API would support to request up to 1000 results around one location (instead 3x20 ), we prefer a grid-based search around multiple search centers (similar as we did with Google Places earlier), which is supposed to provide a higher coverage.

In [22]:
# Define inner and outer quadrangle within we want to search as well as the appropriate step width.
# For the inner quardrangle, we define a smaller step width. 

step = -0.029 # ~2km
smaller_step = step / 2

# Stuttgart area
start_lat = 48.85
end_lat = 48.66
start_long= 9.38
end_long = 8.95

# Stuttgart city
start_lat_city = 48.83
end_lat_city = 48.72
start_long_city = 9.25
end_long_city = 9.10


In [23]:
# Show search centers as markers on Google Maps

maps.configure(PLACES_API_KEY)
fig = maps.figure()

# Draw red markers for the wider area of Stuttgart
marker_locations=[]
for i in np.arange(start_lat, end_lat, step):
    for j in np.arange(start_long, end_long, step):
        # draw markers only if i and j is not within the city area
        if ((i > start_lat_city) | (i < end_lat_city) | (j > start_long_city) | (j < end_long_city)):
            marker_locations.append((i, j))
            
print("Number of red markers: {}".format(len(marker_locations)))   
markers_red = maps.symbol_layer(marker_locations, fill_color='blue', stroke_color='blue')
fig.add_layer(markers_red)

# Draw blue markers for the city area of Stuttgart
marker_locations=[]
for i in np.arange(start_lat_city, end_lat_city, smaller_step):
    for j in np.arange(start_long_city, end_long_city, smaller_step):
        marker_locations.append((i, j))

print("Number of blue markers: {}".format(len(marker_locations)))   
markers_blue = maps.symbol_layer(marker_locations, fill_color='purple', stroke_color='purple')
fig.add_layer(markers_blue)

fig


Number of red markers: 85
Number of blue markers: 88


Figure(layout=FigureLayout(height='420px'))

In [251]:
yelp_ids = []
yelp_names = []
yelp_lats = []
yelp_lngs = []
for marker in marker_locations:
    yelp_result = yelp_api.search_query(
        term='restaurant', latitude=marker[0], longitude=marker[1], radius=1500, limit=50)
    for i in range(len(yelp_result['businesses'])):
        yelp_id = yelp_result['businesses'][i]['id']
        name = yelp_result['businesses'][i]['name']
        lat = yelp_result['businesses'][i]['coordinates']['latitude']
        lng = yelp_result['businesses'][i]['coordinates']['longitude']
        yelp_ids.append(yelp_id)
        yelp_names.append(name)
        yelp_lats.append(lat)
        yelp_lngs.append(lng)
yelp_results.append(yelp_result)


In [254]:
assert(len(yelp_names) == len(yelp_ids) & len(yelp_ids) == len(yelp_lats) & len(yelp_lats) == len(yelp_lngs))

In [265]:
# Build DataFrame
d = {'yelp_id':yelp_ids, 'name':yelp_names, 'latitude': yelp_lats, 'longitude':yelp_lngs}
yelp_restaurants = pd.DataFrame(d)
yelp_restaurants.head()

,yelp_id,name,latitude,longitude
0,-k0Ksk_FSzQAiKU6tdjayg,Restaurant Tsv Steinhaldenfeld,48.829740,9.240350
1,5djuZfIkqf1ykQGbpWESrg,Stadio,48.831900,9.252310
2,R93PaCrS4qai9u1uiiHPMw,"Restaurant ""La Perla"" im Schießsportzentrum",48.833358,9.249487
3,3e_HLr7Qu6Azg7Hp6EK3aw,Rems La Pizza,48.840622,9.267763
4,kscS_CV9KX-j1zdfvBu7mQ,Il Carrettino,48.833700,9.264480


In [268]:
yelp_restaurants['yelp_id'].nunique()

823

In [269]:
# De-dupliacte
yelp_restaurants = yelp_restaurants.drop_duplicates()

#### Yelp IDs found via Matching Google results vs. Yelp IDs found via Yelp directly

`yelp_ids_based_on_google` (IDs based on Google search): 
 * Based on the list of Google IDs, obtained by multiple Google Places Nearby Searches with varying search centers, returning 3x20 results each
 * Based on the Google ID, lookup the Google name and try to find a match at Yelp
 * For 1333 Google restaurants, only 430 matches could be identified, probalby because of name or geolocation mismatch between Yelp and Google (e.g. "Eat Drink Man Woman" vs. "Gasthaus Eat Drink Man Woman")

`yelp_ids_based_on_yelp` (IDs based on Yelp search): 
* Based on multiple Yelp Search Queries with 50 results each, using the same search grid as earlier for the Google search
* 823 results after de-duplication

`yelp_ids_based_on_google` - `yelp_ids_based_on_yelp`: 
* 97 restaurants that are in `yelp_ids_based_on_google`, but not in `yelp_ids_based_on_yelp` .
* Restaurants that were originally found via Google grid search, then mapped to the Yelp equivalents, but are not found via Yelp grid search

`yelp_ids_based_on_yelp` - `yelp_ids_based_on_google`: 
* 498 restaurants that are in `yelp_ids_based_on_yelp`, but not in `yelp_ids_based_on_google` 
* Restaurants that are found via Yelp grid search, but not included in the Yelp equivalents for our previous Google search
* Mostly due to 'name mismatch' between Google name and Yelp name 

==> We continue with a merged set (`yelp_ids_based_on_google` + `yelp_ids_based_on_yelp`) to avoid missing any relevant restaurant, wether it comes from Google or from Yelp. This gives us **823 + 97 = 920 restaurants in the area of Stuttgart**.

In [290]:
yelp_set = set(yelp_ids_based_on_yelp)
google_set = set(yelp_ids_based_on_google)
print(len(yelp_set), len(google_set))

823 422


In [336]:
# IDs that are included in Yelp-based grid search, but not in Google-based grid search
# This includes also the name mismatch cases
yelp_minus_google = yelp_set.difference(google_set)
len(yelp_minus_google)

498

In [335]:
# IDs that are included in Google-based search, but not in Yelp based search
google_minus_yelp = google_set.difference(yelp_set)
len(google_minus_yelp)

97

In [327]:
print("IDs contained in yelp-based but not in google-based search (mostly due to name mismatch)")
for id in yelp_minus_google:
    print(get_name(id))

IDs contained in yelp-based but not in google-based search
The Gardener’s Nosh
Ristorante Le Rose
Bäckerhaus Veit Café
Ristorante Pizzeria La Taverna da Enzo
Weinwirtschaft | Weingut Franz Keller
Zum Adler Gasthaus Weinstube
ESV Rot-Weiss Vereinsgaststätte
Restaurant Schmidener Eintracht
Bäckerhaus Veit Café
Mikail's Lunchbox
Da Mamma Maria
1/1
Yakiniku
Due Stanze e Cucina
Rhythms Bar & Kitchen
Goldener Adler
List
Ristorante Russo Wine Food and Eventlocation
Zum Becher
Gasthof Riedsee
Gasthaus zum Kreuz
Dam Dam Hot Pizza
SEN Asia Soul Kitchen
Cannstatter Tor
Gaststätte Post
Fischlounge
Geiler
Sattlerei
Il Terrazzino
Weinstube Kochenbas
Gaststätte Trompetle
Gravity
Triple B
Cafe & Bar Nion
Taverna Sultan Saray
ALATURKA - Das Stuttgarter Original
Kwan Kao
Noir Cuisine&Bar
[m]eatery
Campioni
Christo's Stube
Funfo's
Bahtti Super Pizza Service
Weinhalle 1896
Cavallino Spaghettaro
Diogenes Taverne
Flori & Palma
Vereinsgaststätte Himmerreich
Los Locos Latinos
Da Micci
Restaurant LE
le tonneau

In [332]:
print("IDs contained in google-based but not in yelp-based search")
for id in google_minus_yelp:
    print(get_name(id))

IDs contained in google-based but not in yelp-based search
Tokyo Sushi
Sultan Saray
Bocca Buona
Hotel Schmidener Eintracht
Cafe Le Theatre
Hotel Hirsch
Ristorante Positano
PAULANER am alten Postplatz
Gaststätte Trollinger
Arcotel Camino
Burger King
Schnauferl Grill
Burger House
Pizzeria Da Cesare
Le Cassoulet
Sultan Saray
Ebony
Aral
La Bruschetta
Amici
Art Sushi
Oggi
Icebreaker Bar im Hotel Pullman Fontana
Indian Palace
Hotalo
Pizza Fiesta
Ristorante La Piazza
Mauritius
Da Noi Due
Aral
JET
Logo - Rotisserie
La Commedia
Enchilada
Aral Tankstelle
Da Franco
Ceylonas
Spielbank Stuttgart
Mezzogiorno
SI-SUITES
Alte Kanzlei
Hotel Neuwirtshaus
Sausalitos
Bella Pizza Service
Eberhard Braun - Gourmetcook - Kochveranstaltungen
Claus
Akzent Hotel Möhringer Hof
5
The Dubliner
Indian Shop
1893 - Das VfB Clubrestaurant
DORMERO Hotel Stuttgart
Im Eimer
Laurita Pizza Service
Früchtle vom Schmidener Feld
Die Palette
Thai Corner
China-Imbiss-Panda
Weber
Pullman Stuttgart Fontana
Cafe Monroe's
Sakura
Lagu

In [361]:
def get_name(yelp_id):
    yelp_restaurant = yelp_api.business_query(id=yelp_id)
    return yelp_restaurant['name']
    

In [350]:
# Write IDs from Yelp-based search to file
df = pd.DataFrame(yelp_ids_based_on_yelp, columns=['yelp_id'])
df.to_csv('yelp_ids_based_on_yelp_search.csv', index=False)
df.to_excel('yelp_ids_based_on_yelp_search.xlsx', index=False)

In [351]:
# Save merged IDs from Google and Yelp-based search 
df = pd.DataFrame(list(merged_ids), columns=['yelp_id'])
df.to_csv('yelp_ids.csv', index=False)
df.to_excel('yelp_ids.xlsx', index=False)

### Enrich merged IDs with detail information

We now have a list of ~820 Yelp IDs, retrieved either by matching the Google Places to Yelp Businesses, or by doing Yelp searches directly. For these IDs, we need to gather the required information (mostly from Yelp). We perform the following steps: 
- For each Yelp ID, make a Yelp business request and enrich the given ID with detail information (such as name, categories, geolocations).
- For each Yelp ID, find the corresponding Google ID from Google Places. This is done with a Google Places find_place search based on Yelp name and location. For every Yelp ID, the corresponding Google ID can be found.

In [465]:
# Read in restaurant IDs.
df_yelp_ids = pd.read_csv('yelp_ids.csv')

In [466]:
# For each Yelp ID, get detail information from Yelp

id_list = []
name_list = []
is_closed_list = []
categories_list = []
price_list = []
rating_list = []
review_count_list = []
photos_list = []
address_list = []
lat_list = []
lng_list = []
for yelp_id in df_yelp_ids['yelp_id'].values:
    restaurant = yelp_api.business_query(id=yelp_id)
    name = restaurant['name']
    is_closed = restaurant['is_closed']
    categories = restaurant['categories']
    categories_unstacked = []
    for c in categories:
        categories_unstacked.append(c['title'])
    if 'price' in restaurant:
        price_level = restaurant['price']
    else:
        price_level = ''
    rating = restaurant['rating']
    review_count = restaurant['review_count']
    if 'photos' in restaurant:
        photo_urls = restaurant['photos']
    else: photo_urls = ''
    address = restaurant['location']['display_address']
    lat = restaurant['coordinates']['latitude']
    lng = restaurant['coordinates']['longitude']
    # Append everything to the lists
    id_list.append(yelp_id)
    name_list.append(name)
    address_list.append(address)
    is_closed_list.append(is_closed)
    categories_list.append(categories_unstacked)
    price_list.append(price_level)
    rating_list.append(rating)
    review_count_list.append(review_count)
    photos_list.append(photo_urls)
    lat_list.append(lat)
    lng_list.append(lng)
    
assert(len(id_list) == len(name_list) & len(name_list) == len(address_list) & len(address_list) == len(is_closed_list)
       & len(is_closed_list) == len(categories_list) & len(categories_list) == len(price_list) & 
       len(price_list) == len(rating_list) & len(rating_list) == len(review_count_list) & 
       len(review_count_list) == len(photos_list) & len(photos_list) == len(lat_list) & 
       len(lat_list) == len(lng_list))

In [473]:
d = {
    'yelp_id' : id_list,
    'yelp_name' : name_list,
    'address' : address_list, 
    'is_closed' : is_closed_list,
    'categories' : categories_list,
    'price_level' : price_list,
    'avg_rating' : rating_list,
    'review_count' : review_count_list,
    'photo_urls' : photos_list,
    'latitude' : lat_list, 
    'longitude' : lng_list
}

df_yelp = pd.DataFrame(d)

In [474]:
#Filter out all restaurants that are  permanently closed -> before 920, afterwards 915
df_yelp.drop(df_yelp[df_yelp['is_closed']==True].index, inplace=True)

In [476]:
# Write details DataFrame to file
#df_yelp.to_csv('yelp_restaurants.csv', index=False)
#df_yelp.to_excel('yelp_restaurants.xlsx', index=False)

In [516]:
# For each Yelp ID, find the corresponding Google ID
# Do so by first looking up the Yelp name, then make a Google Places find_place search based on Yelp name and location

google_ids = []
google_names = []
yelp_name_list = df_yelp['yelp_name'].values

for yelp_name in yelp_name_list:
    google_restaurant = gmaps.find_place(
        input=yelp_name, input_type='textquery', 
        location_bias='point:' + str(lat_hbf) + ', ' + str(lng_hbf), 
        fields=['place_id', 'name'],
        language='de')
    print(yelp_name)
    if (google_restaurant['candidates']):
        google_id = google_restaurant['candidates'][0]['place_id']
        google_name = google_restaurant['candidates'][0]['name']
        print('...OK')
    else:
        google_id = ''
        google_name = ''
        print('...not found')
    google_ids.append(google_id)
    google_names.append(google_name)
        
assert(len(yelp_name_list) == len(google_ids) & len(google_ids) == len(google_names))

Bäckerhaus Veit Café
...OK
Prince of India
...OK
Bocca Buona
...OK
Weinwirtschaft | Weingut Franz Keller
...OK
Zum Adler Gasthaus Weinstube
...OK
ESV Rot-Weiss Vereinsgaststätte
...OK
Restaurant Schmidener Eintracht
...OK
Bäckerhaus Veit Café
...OK
Mikail's Lunchbox
...OK
India House
...OK
Da Mamma Maria
...OK
Ruppmannschänke
...OK
Schnauferl Grill
...OK
Goldener Adler
...OK
List
...OK
Taverne Diogenes
...OK
Wienerwald
...OK
Joe Peña's
...OK
Tschechen & Söhne
...OK
Dam Dam Hot Pizza
...OK
SEN Asia Soul Kitchen
...OK
Cannstatter Tor
...OK
Gaststätte Post
...OK
Kebap Stern
...OK
Geiler
...OK
Art Sushi
...OK
Wirtshaus zum Alten Bahnhof
...OK
Don Giovanni Ristorante Pizzeria
...OK
Cafe & Bar Nion
...OK
ALATURKA - Das Stuttgarter Original
...OK
Kwan Kao
...OK
Gasthof Franziska
...OK
Akeno
...OK
Noir Cuisine&Bar
...OK
Ristorante La Piazza
...OK
Christo's Stube
...OK
Bahtti Super Pizza Service
...OK
Cavallino Spaghettaro
...OK
Flori & Palma
...OK
Vereinsgaststätte Himmerreich
...OK
Restaurant

Nannina
...OK
Das Reiskorn
...OK
Feuerbacher Wichtel
...OK
Indian Shop
...OK
Hallo Emil
...OK
Brauereigaststätte Dinkelacker
...OK
Weinstube Am Stadtgraben
...OK
Ruben's
...OK
Galerie
...OK
Buschpilot
...OK
Die Palette
...OK
Nachtschicht
...OK
McDonald's
...OK
Christophorus
...OK
SGU Gaststätte Delphi
...OK
Cafe Wahnsinn
...OK
El Seco
...OK
Palazzo
...OK
QQ-Sushi Lounge
...OK
Distelfarm
...OK
Sultan Saray
...OK
Hegel Eins
...OK
Weinstube Viertelesschlotzer
...OK
Pizza & Pasta - San Marco
...OK
Ristorante La Scala
...OK
IL Cappuccino
...OK
POP's
...OK
Chicken Tom
...OK
Classic Rock Cafe
...OK
Kursaal Cannstatt
...OK
Weinstube Träuble
...OK
Aral
...OK
Restaurant Filoxenia
...OK
Chito Gvrito
...OK
Der Foehrich
...OK
Vier Burgen
...OK
Aslan Kebab
...OK
Restaurant Ochsen
...OK
Hirsch Gehrung
...OK
Merci
...OK
Udo-Snack
...OK
Zum Spätzleschwob
...OK
Sen Viet
...OK
Red Grill
...OK
Der Zauberlehrling
...OK
pizzabar
...OK
Piri Reis Restaurant & Cafe
...OK
Taverne Saloniki
...OK
KFC
...OK
Imbiss

Cannstatter Kebap Haus
...OK
Flora & Fauna
...OK
Cafebar Auszeit
...OK
Vereinsheim Gartenfreunde Heslach
...OK
Taverna Yol
...OK
Hans Im Glück
...OK
Restaurant Tsv Steinhaldenfeld
...OK
87 Barbecue & Bar
...OK
Reiterstüble Gasthaus
...OK
Restaurant Steinhalde
...OK
Wienerwald
...OK
Tagesbar
...OK
The Burger Republic
...OK
Restaurant Hermes
...OK
Seyffer's
...OK
Aral
...OK
Ristorante La Pergola
...OK
Burreatos
...OK
Gaststätte Ritterstüble
...OK
Trattoria Vivaldi
...OK
NLV Pizzeria
...OK
Jose y Joséfina
...OK
Cafe anno 1897
...OK
la bruschetta
...OK
Dedemoğlu
...OK
peperoncino
...OK
Sportrestaurant Neuwirtshaus
...OK
Restaurant Landhausstuben
...OK
Da Domenico
...OK
Hasen
...OK
Mögle
...OK
Injeera
...OK
DLRG Gaststätte
...OK
Max-Eyth-See
...OK
Gasthaus Rössle
...OK
Plieninger Most- und Weinbesen
...OK
Sapori d'Italia
...OK
Ilysia am Wallgraben
...OK
Persian Restaurant
...OK
Charisma Bootshaus
...OK
Osteria
...OK
Asia Imbiss Hop Sing
...OK
Rosenau - Lokalität & Bühne
...OK
Er Vaquita
...

In [518]:
df_yelp['google_id'] = google_ids
df_yelp['google_name'] = google_names

In [528]:
# Copy and re-order columns. 
df_yelp_reordered = df_yelp[['yelp_id', 'google_id', 'yelp_name', 'google_name', 'address', 'categories', 
                  'price_level', 'avg_rating', 'review_count', 'photo_urls', 'latitude', 'longitude']]
df_yelp = df_yelp_reordered

In [529]:
df_yelp.head()

,yelp_id,google_id,yelp_name,google_name,address,categories,price_level,avg_rating,review_count,photo_urls,latitude,longitude
1,6XKiHruNrTO8jDAcHLoR3w,ChIJF5m2rSLFmUcRH9sBPHwx3kM,Bäckerhaus Veit Café,Bäckerhaus Veit Café,"[Epplestr. 1 b, 70597 Stuttgart, Germany]","[Bakeries, Cafes]",,1.5,5,[https://s3-media2.fl.yelpcdn.com/bphoto/M0vpn...,48.749109,9.169810
2,0Ep-rCF4UVrz3ZJkd_gAXQ,ChIJOWEUrcvEmUcRqRXjZal8AZY,Prince of India,Prince of India,"[Urbanstr. 38, 70182 Stuttgart, Germany]",[Indian],€€,3.0,27,[https://s3-media4.fl.yelpcdn.com/bphoto/rwxyn...,48.778664,9.187046
3,MVScl0YGy_ZkGb7iN58k-A,ChIJI_1zBlHbmUcROMi0FydapcE,Bocca Buona,Bocca Buona Restaurant,"[Hauptstätter Str. 147, 70180 Stuttgart, Germany]",[Italian],,2.5,3,[https://s3-media4.fl.yelpcdn.com/bphoto/NA4u_...,48.764643,9.169958
4,KbIoL9bMYzCZDWxpWTcx0g,ChIJ6wNLZzPbmUcRrtanMZs8deo,Weinwirtschaft | Weingut Franz Keller,Weinwirtschaft | Weingut Franz Keller,"[Schillerstr. 23, 70173 Stuttgart, Germany]","[International, German, Wine Bars]",€€,3.5,10,[https://s3-media2.fl.yelpcdn.com/bphoto/l0icD...,48.782600,9.182460
5,LlLDMhhPeCKs4oyZxFQGaw,ChIJo1Y9RTjQmUcRnsk3LcgHomE,Zum Adler Gasthaus Weinstube,Hotel & Restaurant Adler,"[Alte Stuttgarter Str. 2, 70195 Stuttgart, Ger...",[Restaurants],€,5.0,3,[https://s3-media2.fl.yelpcdn.com/bphoto/6_MBr...,48.778200,9.124740


In [ ]:
# Drop column 'google_name', as it may not be saved permanently
df_yelp = df_yelp.drop(columns=['google_name'])

In [530]:
# Write to file
df_yelp.to_csv('yelp_restaurants.csv', index=False)
df_yelp.to_excel('yelp_restaurants.xlsx', index=False)